#### Estratto dal progetto di TMS:

In [ ]:
import pandas as pd
import re
import nltk
from nltk.stem.porter import PorterStemmer
import string

In [ ]:
returns = re.compile(r"\n")
multiple_spaces = re.compile(r"\s+")

# pre-defined stop words
stop_words = nltk.corpus.stopwords.words('italian') 
porter = PorterStemmer()

In [ ]:
def clean_text(full_text, returns = returns, multiple_spaces = multiple_spaces):

    text = returns.split(full_text)
    new_text = text.lower() # make lowercase
    new_text = new_text.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation))) # replace punctuation with a whitespace
    new_text = multiple_spaces.sub(" ", new_text) # remove multiple whitespaces
    return new_text

def stopwords_remover(lyric, stop_words = stop_words):
    new_text = []
    for token in new_text:
        if token not in stop_words:
            new_text.append(token)
    return new_text
 

#### Estratto dal progetto di DATAMAN:

In [2]:
import pandas as pd
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
import spacy

In [3]:
df_tweets=pd.read_csv('tweets2_nogreenpass_vaccinatevi_casapound.csv')

In [4]:
df_tweets.columns

Index(['Unnamed: 0', 'date', 'id_x', 'user_id', 'text_x', 'author', 'retweet',
       'id_retweet', 'user_id_retweet', 'text_complete', 'hashtags',
       'colours'],
      dtype='object')

In [5]:
df_retweeted=pd.read_csv('retweetted2_nogreenpass_vaccinatevi_casapound.csv')

In [6]:
df_retweeted.columns

Index(['Unnamed: 0', 'date', 'id', 'author_id', 'text', 'like', 'n_rt',
       'author', 'colours'],
      dtype='object')

In [7]:
retweeted=df_retweeted.merge(df_tweets[['id_retweet','user_id_retweet','text_complete','hashtags','colours']],left_on=['id','author_id'],right_on=['id_retweet','user_id_retweet'],how='left')[['id', 'author_id', 'author','n_rt','text_complete','hashtags', 'colours_y']].reset_index(drop=True)

In [8]:
retweeted=retweeted.drop_duplicates(['id','author_id','author']).reset_index(drop=True)

In [9]:
retweeted.columns

Index(['id', 'author_id', 'author', 'n_rt', 'text_complete', 'hashtags',
       'colours_y'],
      dtype='object')

In [109]:
retweeted

,id,author_id,author,n_rt,text_complete,hashtags,colours_y
0,1476028900312551426,2963478049,singerindubai,1,Ho appena visto #DontLookUp incredibile che lo...,['#NoGreenPass'],#fb1239
1,1475926285365006345,1364298062919794688,Lorenzo62752880,106,Anche il sindaco di Verona positivo nonostante...,['#NoGreenPass'],#fb1239
2,1476117524446130180,1296390431190679553,1nessuno100mil2,1,"In Italia si respira una tensione enorme, i n...",['#NoGreenPass'],#fb1239
3,1475960153648291841,1238800491632607232,gianluca826,60,Bassano del Grappa non si ferma. Resistenza do...,['#NoGreenPass'],#fb1239
4,1476095815525584896,1453282570582568960,IlDisilluso,19,"""Sono stati riportati più effetti avversi per ...",[],#aaaacc
...,...,...,...,...,...,...,...
555,1473913473734258688,1028235695092051968,MarcoDiCastro1,2,E qualcuno dirà che adesso non è il momento. M...,['#casapound'],#042069
556,1469698310424477696,215699843,CasaPoundItalia,113,"Super green pass, diversi flashmob di #CasaPou...",[],#aaaacc
557,1470319310551851015,215699843,CasaPoundItalia,42,Video dei blitz di #CasaPound della giornata d...,[],#aaaacc
558,1471763819693318151,215699843,CasaPoundItalia,15,Alcune delle iniziative in programma questo fi...,[],#aaaacc


In [10]:
nlp = spacy.load('it_core_news_lg') #, disable=['parser', 'ner'])

In [11]:
retweeted['text_preprocessed']=retweeted['text_complete'].apply(lambda x: " ".join(x.lower() for x in x.split()))

#df_2['new_text_1']=0
#df_2=df_2.reset_index(drop=True)

In [12]:
#pulizia da simboli
new_text=[]
for index,row in retweeted.iterrows():
    l1=[ t for t in row['text_preprocessed'].split() if t.startswith('#') ]
    l2=[ t for t in row['text_preprocessed'].split() if t.startswith('http') ]
    l3=[ t for t in row['text_preprocessed'].split() if t.startswith('@') ]
    stopword=l1+l2+l3
    querywords=row['text_preprocessed'].split()
    resultwords=[word for word in querywords if word.lower() not in stopword]
    result = ' '.join(resultwords)
    new_text.append(result)

retweeted['text_preprocessed']=new_text

In [60]:
retweeted['text_preprocessed'][0]

'ho appena visto incredibile che lo abbiano permesso:il personaggio filantropo ricorda b. gates/biden e i nuovi ricchi che spacciano le loro idee per scienza.da vedere subito. rispecchia lo stato attuale!'

In [13]:
#vengono cancellati tutti i termini che non sono alfanumerici o che non corrispondono a spazi vuoti
retweeted['text_preprocessed'] = retweeted['text_preprocessed'].str.replace('[^\w\s]',' ')


C:\Users\Eric\AppData\Local\Temp/ipykernel_27632/2839750736.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  retweeted['text_preprocessed'] = retweeted['text_preprocessed'].str.replace('[^\w\s]',' ')


In [14]:
retweeted['text_preprocessed'][0]

'ho appena visto incredibile che lo abbiano permesso il personaggio filantropo ricorda b  gates biden e i nuovi ricchi che spacciano le loro idee per scienza da vedere subito  rispecchia lo stato attuale '

In [63]:
##pulizia da emoji e lemmatization usando la library Spacy
#result=[]
#for i in retweeted.index:
#    stopword=['u001f600','u0001f64f','u0001f5ff','u0001f300','u0001f680','u0001f6ff','u0001f1e0','u0001f1ff','u00002702','u000027b0','u000024c2','u0001f251']
#    querywords=retweeted['text_preprocessed'][i].split()
#    resultwords=[word for word in querywords if word.lower() not in stopword]
#    result.append(' '.join(resultwords))
#    
#retweeted['text_preprocessed']=result
#
#stop = stopwords.words('italian')
#
#retweeted['text_preprocessed'] = retweeted['text_preprocessed'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))


In [64]:
retweeted['text_preprocessed'][0]

'appena visto incredibile permesso personaggio filantropo ricorda b gates biden nuovi ricchi spacciano idee scienza vedere subito rispecchia stato attuale'

In [15]:
def space(comment):
    doc = nlp(comment)
    return " ".join([token.lemma_ for token in doc])

retweeted['text_preprocessed']= retweeted['text_preprocessed'].apply(space)

In [15]:
retweeted['text_preprocessed'][0]

'avere appena vistare incredibile che lo avere permettere il personaggio filantropo ricordare b   gates biden e il nuovo ricco che spacciare il loro idea per scienza da vedere subito   rispecchiare il stato attuale'

In [16]:
from nltk.sentiment import vader
import nltk

In [21]:
sentix=pd.read_csv("sentix.csv", header=None,delimiter=';')

In [133]:
#riscaliamo i valori della sentiment come i precedenti valori del lexicon vader
#consideriamo valori estremi di vader lexicon -4 e +4
#consideriamo valori estremi di sentix lexicon -1 e +1

with open("sentix.txt",'w',encoding="utf-8") as txt_file:
    for index,line in sentix.iterrows():
        scaled_score=(((float(line[5]) + 1)/2)*(8))-4
        txt_file.write(str(line[0])+'\t'+str(scaled_score)+'\n')

In [96]:
sentix.head()

,0,1,2,3,4,5,6
0,abile,a,1740,0.125,0.00,1.0,0.125
1,intelligente,a,1740,0.125,0.00,1.0,0.125
2,valente,a,1740,0.125,0.00,1.0,0.125
3,capace,a,1740,0.125,0.00,1.0,0.125
4,incapace,a,2098,0.000,0.75,-1.0,0.750


In [9]:
set(list(sentix.iloc[:,1]))

{'a', 'n', 'r', 'v'}

In [22]:
d= dict([(parola,score) for parola, score in zip(sentix[0],sentix[5])])
sentix_dict=d

In [23]:
Analyzer = vader.SentimentIntensityAnalyzer()

Analyzer.lexicon.update(sentix_dict)

In [24]:
from vaderSentiment import vaderSentiment

In [143]:
#arricchimento vader lexicon
with open('lexicon_vader_enriched.txt','w',encoding="utf-8") as newfile:
    with open('vader_lexicon.txt') as vader_file:
        for line in vader_file:
            newfile.write(line)
    with open('sentix.txt',encoding="utf-8") as sentix_file:
        for line in sentix_file:
            newfile.write(line)

In [25]:
from vader_sentiment.vader_sentiment import SentimentIntensityAnalyzer

In [26]:
analyzer2=SentimentIntensityAnalyzer(lexicon_file='sentix.txt',emoji_lexicon='emoji_utf8_lexicon.txt')

In [27]:
#non facciamo pulizia del testo sulle emojii
analyzer = vaderSentiment.SentimentIntensityAnalyzer(lexicon_file='lexicon_vader_enriched.txt',emoji_lexicon='emoji_utf8_lexicon.txt')

#il termine "positivo" è stato modificato associandolo a un sentiment negativo
#il termine "negativo" è associato a un sentiment neutro


In [46]:
testo=retweeted['text_preprocessed'][16]
#testo='positivo'

print(testo)
print('\n')
print('NLTK: ',Analyzer.polarity_scores(testo))
print('VADER-MULTI: ', analyzer.polarity_scores(testo)) #algoritmo migliore
print('VADER: ', analyzer2.polarity_scores(testo))

2022   il futuro essere soltanto nostro il coord   emilia rom   invitare tutto il cittadino venerdì 31 dicembre   alla ora 15 00   presso piazzale jacchia del giardino margherita di per celebrare il nuovo anno in uno evento comporre da parola   teatro   poesia e musica


NLTK:  {'neg': 0.141, 'neu': 0.572, 'pos': 0.286, 'compound': 0.6798}
VADER-MULTI:  {'neg': 0.101, 'neu': 0.791, 'pos': 0.108, 'compound': -0.1779}
VADER:  {'neg': 0.211, 'neu': 0.359, 'pos': 0.43, 'compound': 0.9655}
